With topic modeling complete for the vanilla, coded, and valence coded review sets, we can now compare and visualize the outputs to see which inputs were most successful for topic modeling. 

In [23]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
#import re
#import os
%matplotlib inline

In [14]:
df = pd.read_csv('amazon_reviews_clean.csv')
df.set_index('Id', inplace=True, verify_integrity=True)
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 
                'clean_valenc'], inplace=True)
df.head(2)

,Unnamed: 0.1.1.1.1,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,...,clean_coded,clean_valenc,clean_valence,codecount_GOOD,codecount_BAD,valencecount_GOOD,valencecount_VGOOD,valencecount_BAD,valencecount_VBAD,Sentiment
Id,,,,,,,,,,,,,,,,,,,,,
14,13,B001GVISJM,A18ECVX2RJ7HUE,"willie ""roadie""",2,2,4,1288915200,fresh and greasy!,good flavor! these came securely packed... the...,...,good GOODREVIEW flavor come securely GOODREVIE...,"['good', 'GOODREVIEW', 'flavor', '!', 'come', ...",good GOODREVIEW flavor come securely GOODREVIE...,5,0,5,2,0,0,1
15,14,B001GVISJM,A2MUGFV2TDQ47K,"Lynrie ""Oh HELL no""",4,5,5,1268352000,Strawberry Twizzlers - Yummy,The Strawberry Twizzlers are my guilty pleasur...,...,strawberry twizzlers be guilty BADREVIEW pleas...,"['strawberry', 'twizzlers', 'be', 'guilty', 'B...",strawberry twizzlers be guilty BADREVIEW pleas...,1,1,1,1,1,0,1


In [26]:
# other helper functions
from gensim import corpora
import gensim, logging, io, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
handler = logging.FileHandler('training_output.log')

def remove_freq(dictionary, n):
    logger = logging.getLogger('gensim.corpora.dictionary')
    save = set(['GOODREVIEW', 'BADREVIEW', 'VGOODREVIEW', 'VBADREVIEW'])
    most_frequent_ids = (v for v in (dictionary.token2id).values() if dictionary[v] not in save)
    most_frequent_ids = sorted(most_frequent_ids, key=dictionary.dfs.get, reverse=True)
    most_frequent_ids = most_frequent_ids[:n]
    # do the actual filtering, then rebuild dictionary to remove gaps in ids
    most_frequent_words = [(dictionary[idx], dictionary.dfs.get(idx, 0)) for idx in most_frequent_ids]
    logger.info("discarding %i tokens: %s...", len(most_frequent_ids), most_frequent_words[:10])

    dictionary.filter_tokens(bad_ids=most_frequent_ids)
    logger.info("resulting dictionary: %s", dictionary)

In [7]:
review_counts = df['ProductId'].value_counts().sort_values()
top_ten = review_counts.tail(10).index.values

In [44]:
# import the topic modeling results sets
vanilla_final_results = pd.read_csv('vanilla_final_results.csv')
vanilla_final_results.set_index('product', inplace=True)
final_results = pd.read_csv('final_results.csv')
final_results.set_index('product', inplace=True)
valence_final_results = pd.read_csv('valence_final_results.csv')
valence_final_results.set_index('product', inplace=True)

## PyLDAVis Visualizations


## Bigrams Visualization


In [1]:
from nltk.collocations import BigramAssocMeasures, TrigramAssocMeasures, QuadgramCollocationFinder, BigramCollocationFinder, TrigramCollocationFinder
from nltk import word_tokenize, FreqDist, bigrams
import operator
import pickle

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet, Jitter, HoverTool
from bokeh.palettes import GnBu3, OrRd3
from bokeh.core.properties import value
from bokeh.transform import dodge, jitter
from bokeh.layouts import gridplot

output_notebook()

with open("trigrams_filter.txt", "rb") as f:
    trigrams_filter = pickle.load(f)
with open("bigrams_filter.txt", "rb") as f:
    bigrams_filter = pickle.load(f)

Loading BokehJS ...

In [ ]:
# TO REFACTOR
# helper functions to visualize topics and bigrams per product


def bigrams_by_sent(df, top_bigram, text_input, topic):  
    bigram_measures = BigramAssocMeasures()
    try:
        words = np.concatenate(np.array([word_tokenize(r) for r in df[text_input].values]))
        bigram_fd = FreqDist(bigrams(words))

        finder = BigramCollocationFinder.from_words(words, window_size=3)
        finder.apply_word_filter(lambda w: w in ('GOODREVIEW', 'BADREVIEW', 'VGOODREVIEW', 'VBADREVIEW', 's',
                                                    'b', 'c', 'oz', 'be'))

        finder.apply_freq_filter(3)
        finder.apply_ngram_filter(lambda w1, w2: (w1, w2) in bigrams_filter)
        top_bigrams = finder.nbest(bigram_measures.pmi, 6)
        grams=[]
        if top_bigram in top_bigrams:
            for a in top_bigrams:
                if a != top_bigram:
                    grams.append(a[0]+" "+a[1])
        else:
            grams = [a[0]+" "+a[1] for a in top_bigrams]
        chi_sq_raw = dict(finder.score_ngrams(bigram_measures.chi_sq))
        raw_freq = dict(finder.score_ngrams(bigram_measures.raw_freq))
        pmi_raw = dict(finder.score_ngrams(bigram_measures.pmi))
        # use PMI as a filter - only keep bigrams with a positive PMI score (indicates positive association)
        # use chi_sq as a filter - only want bigrams that pass the significance threshold of 3.841
        pmi = {k:v for (k,v) in pmi_raw.items() if v > 0}
        chi_sq = {k:v for (k,v) in chi_sq_raw.items() if v >3.841}
    except ValueError:
        # this is the case where no reviews in the set were matched with the topic selected
        # so there is no data to add
        grams = []
        pmi = []
        chi_sq = []
        raw_freq = []
    pmi = pd.Series(pmi)
    chi_sq = pd.Series(chi_sq).apply(np.log)
    raw_freq=pd.Series(raw_freq)

    gram_df = pd.concat([pmi, chi_sq, raw_freq], axis=1).reset_index()
    gram_df.rename(columns={0: 'pmi', 1:'chi_sq', 2:'raw_freq'}, inplace=True)
    gram_df['topic'] = "Topic {}".format(topic)
    try:
        gram_df['bigram'] = gram_df['level_0']+" "+gram_df['level_1']
        gram_df['bigram'] = gram_df['bigram'].apply(lambda x: tuple(list(x.split())))
        gram_df = gram_df[['pmi', 'chi_sq', 'raw_freq', 'topic', 'bigram']]
    except KeyError:
        if gram_df.empty:
            gram_df = pd.DataFrame(columns=['pmi', 'chi_sq', 'raw_freq', 'topic','bigram'])
        else:
            print("Other error with building bigrams dataframe.")
    return grams, gram_df

def get_bigrams(product, topic, text_input, top_bigram):

    topic_df = sub_df_ten.loc[(sub_df_ten["ProductId"]==product)&(sub_df_ten['max_topic']==topic)]
    pos_df = topic_df[topic_df['Sentiment']==1]
    neg_df = topic_df[topic_df['Sentiment']==0]
    bigram_measures = BigramAssocMeasures()

    grams_pos, pos_bg_df = bigrams_by_sent(pos_df, top_bigram, text_input, topic)
    pos_bg_df['sent']='positive'
    grams_neg, neg_bg_df = bigrams_by_sent(neg_df, top_bigram, text_input, topic)
    neg_bg_df['sent']='negative'

    return grams_pos, grams_neg, pos_bg_df, neg_bg_df



def vis_topics_sent(product, text_input):
    product_df = sub_df_ten.loc[sub_df_ten['ProductId']==product]
    topic_counts = product_df['max_topic'].value_counts().to_dict()
    topics = ['Topic {}'.format(t) for t in sorted(topic_counts)]
    counts = [topic_counts[t] for t in sorted(topic_counts)]
    product_words = np.concatenate(np.array([word_tokenize(r) for r in product_df[text_input].values]))
    sentiments = ["Positive", "Negative"]
    pos_counts = []
    neg_counts = []
    for t in sorted(topic_counts):
        topic_df = product_df[product_df['max_topic']==t]
        sent_counts = topic_df['Sentiment'].value_counts()
        try:
            pos_counts.append(sent_counts.loc[1])
        except KeyError:
            pos_counts.append(0)
        try: 
            neg_counts.append(sent_counts.loc[0])
        except KeyError:
            neg_counts.append(0)
    
    bigram_fd = FreqDist(bigrams(product_words))
    top_bigram = max(bigram_fd.items(), key=operator.itemgetter(1))[0]
    gram_list_pos = []
    gram_list_neg = []

    bg_df = pd.DataFrame(columns=['topic','pmi', 'chi_sq', 'raw_freq', 'bigram', 'sent'])
    for t in sorted(topic_counts):
        grams_pos, grams_neg, pos_bg_df, neg_bg_df = get_bigrams(product=product, text_input=text_input, 
                                                                 topic=t, top_bigram=top_bigram)
        bg_df = pd.concat([pos_bg_df, bg_df], axis=0)
        bg_df = pd.concat([neg_bg_df, bg_df], axis=0)
        gram_list_pos.append(grams_pos)
        gram_list_neg.append(grams_neg)
    
    colormap = {'negative': 'red', 'positive': 'blue'}

    bg_df['color'] = bg_df['sent'].map(lambda x: colormap[x])
    source2 = ColumnDataSource(data=bg_df) # for right chart
    source1 = ColumnDataSource(data={'topics': topics, # for left chart
           'Positive': pos_counts,
           'Negative': neg_counts,
           'counts': counts,
            'labels_pos': gram_list_pos,
            'labels_neg': gram_list_neg
           })
    
    # build the lefthand chart to show review counts by topic and by review sentiment
    left = figure(y_range=topics, x_range=(0, 1000), plot_height=500, \
               title="Topic Outputs by Review Sentiment for Product {}".format(product), toolbar_location=None, tools="")
    left.hbar(y=dodge('topics', -0.25, range=left.y_range), right='Positive', height=0.2, source=source1,
           color="#718dbf", legend=value("Positive Reviews"))

    left.hbar(y=dodge('topics',  0.0,  range=left.y_range), right='Negative', height=0.2, source=source1,
           color="#e84d60", legend=value("Negative Reviews"))

    left.y_range.range_padding = 0.1
    left.ygrid.grid_line_color = None
    left.legend.location = "bottom_right"
    left.legend.orientation = "vertical"
    
    labels1 = LabelSet(y='topics', x='Positive', text='labels_pos', source=source1, level='glyph', render_mode='canvas',
                     x_offset=5, y_offset=-20, text_font_size="8pt")
    labels2 = LabelSet(y='topics', x='Negative', text='labels_neg', source=source1, level='glyph', render_mode='canvas',
                     x_offset=5, y_offset=0, text_font_size="8pt")
    left.xaxis.axis_label = 'Number of Reviews'

    left.add_layout(labels1)
    left.add_layout(labels2)
    
  
    # set up the righthand chart of the bigrams
    right = figure(y_range=topics, plot_height=500, title="Bigrams by Review Sentiment for Product {}".format(product),
                  toolbar_location=None, tools="")
    # split out and plot negative- and positive-review-associated bigrams separately
    # so they can be hidden via interactive legend
    df_pos = bg_df[bg_df['color']=='blue']
    df_neg = bg_df[bg_df['color']=='red']
    source_pos = ColumnDataSource(df_pos)
    source_neg = ColumnDataSource(df_neg)
    r_pos = right.circle(x='chi_sq', y=jitter('topic', width=0.3, range=right.y_range), fill_color='color', line_color='color',
                 source=source_pos, fill_alpha=0.3, legend='sent', muted_color='color', muted_alpha=0.2,)
    r_neg = right.circle(x='chi_sq', y=jitter('topic', width=0.3, range=right.y_range), fill_color='color', line_color='color',
             source=source_neg, fill_alpha=0.3, legend='sent', muted_color='color', muted_alpha=0.2)
    right.add_tools(HoverTool(renderers=[r_neg], tooltips=[("bigram","@bigram"), ("score","@chi_sq")]))
    right.add_tools(HoverTool(renderers=[r_pos], tooltips=[("bigram","@bigram"), ("score","@chi_sq")]))

    right.xaxis.axis_label = "Log of Chi-Squared Score"
    right.legend.click_policy="hide"
    
    p = gridplot([[left, right]])
    return p



## Topic Coherence Comparison

In [84]:
def print_topic_coherences(product, df, final_results, text_input):
    """
    product: the product code/ID
    df: the full dataframe of reviews 
    final_results: the final results dataframe containing the optimized number of topics and passes
    text_input: the specific review set - vanilla, coded, or valence coded
    file_path: the folder to find the saved LDA topic models
    """
    
    data = df[df['ProductId']==product]
    # prepare the corpus
    texts = data[text_input].str.split()
    dictionary = gensim.corpora.Dictionary(texts)
    # filter out the ten most frequent tokens
    remove_freq(dictionary, 3)
    #corpus = [dictionary.doc2bow(text) for text in texts]
    # load in the tuned LDA model for the product
    t = final_results.loc[product, 'num_topics']
    p = final_results.loc[product, 'passes']
    file_paths = {'clean_words': 'vanilla_outputs', 'clean_coded': 'outputs', 'clean_valence': 'valence_outputs'}
    file_path = file_paths[text_input]
    lda = gensim.models.ldamodel.LdaModel.load("./"+file_path+"/final_models/{}_{}_{}".format(product, t, p))
    #print("Topic Coherences for Product {}".format(product))

    cohs = []
    for n, topic in lda.show_topics(formatted=False):
        #print(n)
        topic = [word for word, _ in topic]
        cm = gensim.models.coherencemodel.CoherenceModel(topics=[topic], texts=texts, 
                                                         dictionary=dictionary, window_size=2, coherence='u_mass')
        coh = cm.get_coherence()
       # print(coh)
        cohs.append([product, n, "Topic {}".format(n), text_input, coh])
    
    cohs = pd.DataFrame(data=cohs, columns=['Product', 't', 'Topic', 'Encoding', 'Coherence'])
    return cohs


In [85]:
topic_coherences = pd.DataFrame(columns=['Product', 't', 'Topic', 'Encoding', 'Coherence'])
for i, f in zip(['clean_words', 'clean_coded', 'clean_valence'], 
                [vanilla_final_results, final_results, valence_final_results]):
    for product in top_ten:
        cohs = print_topic_coherences(product=product, df=df, final_results=f, 
                               text_input=i)
        topic_coherences = pd.concat([topic_coherences, cohs], axis=0, ignore_index=True)

2018-07-16 16:12:04,697 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-16 16:12:04,816 : INFO : built Dictionary(3861 unique tokens: ['ounce', 'chocolate', 'remote', 'mef', 'air']...) from 487 documents (total 35917 corpus positions)
2018-07-16 16:12:04,827 : INFO : discarding 3 tokens: [('coffee', 439), ('is', 417), ('not', 396)]...
2018-07-16 16:12:04,840 : INFO : resulting dictionary: Dictionary(3858 unique tokens: ['ounce', 'chocolate', 'remote', 'mef', 'air']...)
2018-07-16 16:12:04,842 : INFO : loading LdaModel object from ./vanilla_outputs/final_models/B002IEZJMA_10_50
2018-07-16 16:12:04,850 : INFO : loading expElogbeta from ./vanilla_outputs/final_models/B002IEZJMA_10_50.expElogbeta.npy with mmap=None
2018-07-16 16:12:04,859 : INFO : setting ignored attribute id2word to None
2018-07-16 16:12:04,862 : INFO : setting ignored attribute dispatcher to None
2018-07-16 16:12:04,865 : INFO : setting ignored attribute state to None
2018-07-16 16:12:04,867 : INFO

2018-07-16 16:12:10,630 : INFO : loading LdaModel object from ./vanilla_outputs/final_models/B0013A0QXC_6_50
2018-07-16 16:12:10,634 : INFO : loading expElogbeta from ./vanilla_outputs/final_models/B0013A0QXC_6_50.expElogbeta.npy with mmap=None
2018-07-16 16:12:10,639 : INFO : setting ignored attribute id2word to None
2018-07-16 16:12:10,640 : INFO : setting ignored attribute dispatcher to None
2018-07-16 16:12:10,643 : INFO : setting ignored attribute state to None
2018-07-16 16:12:10,644 : INFO : loaded ./vanilla_outputs/final_models/B0013A0QXC_6_50
2018-07-16 16:12:10,646 : INFO : loading LdaState object from ./vanilla_outputs/final_models/B0013A0QXC_6_50.state
2018-07-16 16:12:10,653 : INFO : loaded ./vanilla_outputs/final_models/B0013A0QXC_6_50.state
2018-07-16 16:12:11,074 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-16 16:12:11,188 : INFO : built Dictionary(3831 unique tokens: ['ounce', 'chocolate', 'air', 'worth', 'normally']...) from 556 documents (to

2018-07-16 16:12:16,646 : INFO : loaded ./outputs/final_models/B005ZBZLT4_12_50.state
2018-07-16 16:12:17,426 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-16 16:12:17,553 : INFO : built Dictionary(3164 unique tokens: ['ounce', 'chocolate', 'cuisinart', 'caribbean', 'air']...) from 530 documents (total 47454 corpus positions)
2018-07-16 16:12:17,564 : INFO : discarding 3 tokens: [('coffee', 518), ('be', 509), ('vanilla', 474)]...
2018-07-16 16:12:17,574 : INFO : resulting dictionary: Dictionary(3161 unique tokens: ['ounce', 'chocolate', 'cuisinart', 'caribbean', 'air']...)
2018-07-16 16:12:17,576 : INFO : loading LdaModel object from ./outputs/final_models/B003GTR8IO_12_100
2018-07-16 16:12:17,585 : INFO : loading expElogbeta from ./outputs/final_models/B003GTR8IO_12_100.expElogbeta.npy with mmap=None
2018-07-16 16:12:17,590 : INFO : setting ignored attribute state to None
2018-07-16 16:12:17,592 : INFO : setting ignored attribute id2word to None
2018-07-16 16:

2018-07-16 16:12:22,471 : INFO : setting ignored attribute state to None
2018-07-16 16:12:22,473 : INFO : setting ignored attribute id2word to None
2018-07-16 16:12:22,474 : INFO : setting ignored attribute dispatcher to None
2018-07-16 16:12:22,476 : INFO : loaded ./outputs/final_models/B0026RQTGE_7_70
2018-07-16 16:12:22,477 : INFO : loading LdaState object from ./outputs/final_models/B0026RQTGE_7_70.state
2018-07-16 16:12:22,482 : INFO : loaded ./outputs/final_models/B0026RQTGE_7_70.state
2018-07-16 16:12:23,005 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-16 16:12:23,121 : INFO : built Dictionary(2971 unique tokens: ['ounce', 'chocolate', 'online', 'remote', 'mef']...) from 487 documents (total 41120 corpus positions)
2018-07-16 16:12:23,132 : INFO : discarding 3 tokens: [('be', 478), ('coffee', 444), ('not', 397)]...
2018-07-16 16:12:23,142 : INFO : resulting dictionary: Dictionary(2968 unique tokens: ['ounce', 'chocolate', 'remote', 'mef', 'air']...)
201

2018-07-16 16:12:29,896 : INFO : built Dictionary(2167 unique tokens: ['ounce', 'p', 'introduction', 'past', 'remote']...) from 542 documents (total 22866 corpus positions)
2018-07-16 16:12:29,907 : INFO : discarding 3 tokens: [('be', 486), ('coffee', 434), ('senseo', 318)]...
2018-07-16 16:12:29,915 : INFO : resulting dictionary: Dictionary(2164 unique tokens: ['ounce', 'chocolate', 'past', 'remote', 'ambiance']...)
2018-07-16 16:12:29,919 : INFO : loading LdaModel object from ./valence_outputs/final_models/B0013A0QXC_15_80
2018-07-16 16:12:29,925 : INFO : loading expElogbeta from ./valence_outputs/final_models/B0013A0QXC_15_80.expElogbeta.npy with mmap=None
2018-07-16 16:12:29,931 : INFO : setting ignored attribute state to None
2018-07-16 16:12:29,933 : INFO : setting ignored attribute id2word to None
2018-07-16 16:12:29,934 : INFO : setting ignored attribute dispatcher to None
2018-07-16 16:12:29,939 : INFO : loaded ./valence_outputs/final_models/B0013A0QXC_15_80
2018-07-16 16:12:2

In [95]:

    
def get_cds(df):
    cds = ColumnDataSource(data=df)
    return cds
    
def vis_coherences(product, topic_coherences):
    df = topic_coherences[topic_coherences['Product']==product]
    vanilla = df[df['Encoding']=='clean_words'].sort_values(by="Coherence")
    coded = df[df['Encoding']=='clean_coded'].sort_values(by="Coherence")
    valence = df[df['Encoding']=='clean_valence'].sort_values(by="Coherence")
    
    lsource = get_cds(vanilla)
    csource = get_cds(coded)
    rsource = get_cds(valence)
    
    left = figure(y_range=[-20,0], x_range=vanilla['Topic'].unique(), plot_height=250, plot_width=250, title="Vanilla Reviews Topic Coherence", tools="")
    left.vbar(x="Topic", top='Coherence', width=0.9, source=lsource)
    left.xaxis.major_label_orientation = 1
    
    center = figure(y_range=[-20,0], x_range=coded['Topic'].unique(), plot_height=250, plot_width=250, title="Coded Reviews Topic Coherence", tools="")
    center.vbar(x="Topic", top='Coherence', width=0.9, source=csource)
    center.xaxis.major_label_orientation = 1
    
    right = figure(y_range=[-20,0], x_range=valence['Topic'].unique(), plot_height=250, plot_width=250, title="Valence Coded Reviews Topic Coherence", tools="")
    right.vbar(x="Topic", top='Coherence', width=0.9, source=rsource)
    right.xaxis.major_label_orientation = 1
    
    p = gridplot([[left, center, right]])
    return p

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In using the u_mass topic coherence metric, large negative values mean a topic is composed of words that tend not to occur together. Values closer to zero are better and mean the words ARE co-occurring more frequently. 

In [96]:
for product in top_ten:
    p = vis_coherences(product, topic_coherences)
    show(p)

,Product,t,Topic,Encoding,Coherence
0,B002IEZJMA,0,Topic 0,clean_words,-10.778131
1,B002IEZJMA,1,Topic 1,clean_words,-0.865356
2,B002IEZJMA,2,Topic 2,clean_words,-12.815905
3,B002IEZJMA,3,Topic 3,clean_words,-18.665741
4,B002IEZJMA,4,Topic 4,clean_words,-16.078783
5,B002IEZJMA,5,Topic 5,clean_words,-11.283691
6,B002IEZJMA,6,Topic 6,clean_words,-1.779494
7,B002IEZJMA,7,Topic 7,clean_words,-14.255627
8,B002IEZJMA,8,Topic 8,clean_words,-15.108473
9,B002IEZJMA,9,Topic 9,clean_words,-1.408431
